## New analysis goal: fixed skill sets from tourneys

Use df_matches to identify the group ids that use competitive rulesets.
This gives us the team ids we need.
Then we can get the team roster using https://fumbbl.com/api/team/get/1102662
we need the player ids here.

and the chosen skills using https://fumbbl.com/api/team/getOptions/1102662
this is a string of player ids, with skill ids.

through https://fumbbl.com/api/skill/list we can get a list of skills.
https://fumbbl.com/api/skill/list/2020 and this is the 2020 list.

* World cup training: 9941 (dec 2020), 2 matches in dec 2020 using bb2016, then in okt 2021 for real. 429 matches.
* SUper league: 15615
* templars road to WC: 11605
* entrainment tournois: 12879
* NAF online tournaments : 9298
* Tacklezone: 12013
* Doppelbock: 13198
* Eurobowl practice league: 15643
* Eurobowl 2020 training: 12087 (eurobowl warsaw)

In [ ]:
import pandas as pd
import numpy as np
import plotnine as p9
import requests

from mizani.formatters import date_format

# point this to the location of the HDF5 datasets
path_to_datasets = 'datasets/current/'

# FUMBBL matches
target = 'df_matches.h5'
df_matches = pd.read_hdf(path_to_datasets + target) 



In [ ]:
# FUMBBL inducements
target = 'inducements.h5'
inducements = pd.read_hdf(path_to_datasets + target) 

# FUMBBL skills
target = 'df_skills.h5'
df_skills = pd.read_hdf(path_to_datasets + target) 

In [ ]:
inducements.query('match_id == 4335359')

In [ ]:
%run read_json_file.py
%run write_json_file.py


https://fumbbl.com/api/ruleset/get/188
contains info about gold and skills.

From the roster id we can infer the bb version.

# Gather Team roster info

Need a combi of Team and Match to get the inducements for the roster

In [ ]:
# NAF eurobowl -- EB 2022 - Group 2

team_ids = df_matches.query('tournament_id == 58323')['team1_id'].tolist() + df_matches.query('tournament_id == 58323')['team2_id'].tolist()

team_ids = list(set(team_ids))

In [ ]:
# how many teams?
len(team_ids)

In [ ]:
%run get_team_roster.py


In [ ]:
df_rosters = get_team_roster(team_ids[0], df_skills, df_matches, inducements)

team_ids.pop(0)

for team_id in team_ids:
    df_rosters_tmp = get_team_roster(team_id, df_skills, df_matches, inducements)
    df_rosters = pd.concat([df_rosters, df_rosters_tmp], ignore_index=True)

In [ ]:
df_rosters

In [ ]:
target = 'datasets/current/df_rosters'

#df_rosters.to_hdf(target + '.h5', key='df_rosters', mode='w', format = 't',  complevel = 9)
df_rosters.to_csv(target + '.csv')

# PM add coach and tournament id to roster 